# Teradata Connection Demo

This notebook demonstrates how to connect to a Teradata database using the `TeradataConnection` class with YAML-based configuration for test and prod environments.

## 1. Import Required Libraries

Import necessary libraries for database connection and data processing.

In [1]:
import logging
import sys
from pathlib import Path
from sqlalchemy import text
import pandas as pd

# Add src to path for imports
sys.path.insert(0, str(Path.cwd()))

# Import the connection module
from src.connection import TeradataConnection, TeradataConnectionError

# Configure logging to see connection information
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Configure Connection Parameters

The connection parameters are stored in `td_env.yaml` file which is not committed to git for security reasons. 

Create your own `td_env.yaml` file from `td_env.yaml.template`:

```yaml
test:
  host: "test-teradata-server.company.com"
  username: "your_username"
  password: "your_password"
  database: "test_database"
  logmech: "TD2"
  tmode: "ANSI"
  charset: "UTF8"

prod:
  host: "prod-teradata-server.company.com"
  username: "your_username"
  password: "your_password"
  database: "prod_database"
  logmech: "TD2"
  tmode: "ANSI"
  charset: "UTF8"
```

## 3. Establish Database Connection

Initialize the connection manager and list available environments.

In [2]:
try:
    # Initialize the connection manager
    conn_manager = TeradataConnection()
    print("✓ Connection manager initialized successfully")
    
    # List available environments
    environments = conn_manager.list_environments()
    print(f"✓ Available environments: {environments}")
    
except TeradataConnectionError as e:
    print(f"✗ Connection Error: {e}")
    print("\nPlease ensure:")
    print("1. td_env.yaml file exists in the project root")
    print("2. Copy td_env.yaml.template to td_env.yaml")
    print("3. Update credentials for your test/prod environments")

2026-01-13 20:35:32,245 - src.connection - INFO - Loaded configuration for: ['test', 'prod']


✓ Connection manager initialized successfully
✓ Available environments: ['test', 'prod']


In [3]:
import sqlalchemy as sa
conn = sa.create_engine("teradatasql://@teradw/DBC?logmech=BROWSER")
conn.connect()
conn = sa.create_engine("teradatasql://@teradw/ALL?logmech=BROWSER",
                    pool_size=3,
                    pool_pre_ping=True,  # Verify connections before using
                    echo=True,)
conn.connect()

2026-01-13 20:35:39,310 INFO sqlalchemy.engine.Engine SELECT dbc.dbcinfov."InfoData" 
FROM dbc.dbcinfov 
WHERE dbc.dbcinfov."InfoKey" = ?


2026-01-13 20:35:39,310 - sqlalchemy.engine.Engine - INFO - SELECT dbc.dbcinfov."InfoData" 
FROM dbc.dbcinfov 
WHERE dbc.dbcinfov."InfoKey" = ?


2026-01-13 20:35:39,313 INFO sqlalchemy.engine.Engine [dialect teradatasql+teradatasql does not support caching 0.00253s] ('VERSION',)


2026-01-13 20:35:39,313 - sqlalchemy.engine.Engine - INFO - [dialect teradatasql+teradatasql does not support caching 0.00253s] ('VERSION',)


2026-01-13 20:35:39,356 INFO sqlalchemy.engine.Engine select database


2026-01-13 20:35:39,356 - sqlalchemy.engine.Engine - INFO - select database


2026-01-13 20:35:39,358 INFO sqlalchemy.engine.Engine [dialect teradatasql+teradatasql does not support caching 0.00198s] ()


2026-01-13 20:35:39,358 - sqlalchemy.engine.Engine - INFO - [dialect teradatasql+teradatasql does not support caching 0.00198s] ()


## 4. Test the Connection

Attempt to connect to the test environment and verify the connection is successful.

In [4]:
try:
    # Get engine for test environment
    env_name = 'test'
    engine = conn_manager.get_engine(env_name)
    
    print(f"✓ Successfully connected to '{env_name}' environment")
    print(f"  Engine URL: {engine.url}")
   # print(f"  Connection Pool Size: {engine.pool.size()}")
    
    # Verify connection with a simple query
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1 AS test_value"))
        value = result.scalar_one()
        print(f"✓ Test query result: {value}")
        
except TeradataConnectionError as e:
    print(f"✗ Connection failed: {e}")
except Exception as e:
    print(f"✗ Unexpected error: {e}")

2026-01-13 20:35:39,454 - src.connection - INFO - Connection string: teradatasql://@teradw/ALL?logmech=BROWSER


Connection string: teradatasql://@teradw/ALL?logmech=BROWSER


2026-01-13 20:35:41,042 - src.connection - INFO - Created connection to 'test' environment


✓ Successfully connected to 'test' environment
  Engine URL: teradatasql://@teradw/ALL?logmech=BROWSER
✓ Test query result: 1


## 5. Execute a Sample Query

Use the context manager to execute a query and retrieve data as a pandas DataFrame.

In [5]:
try:
    # Example: Query system information from DBC database
    # This example queries system tables available in most Teradata databases
    
    sample_query = """
    SELECT 
        CAST(CURRENT_DATE AS VARCHAR(10)) AS query_date,
        CAST(CURRENT_TIME AS VARCHAR(8)) AS query_time,
        'test' AS environment_name
    """
    
    # Use context manager for connection
    with conn_manager.get_connection('test') as engine:
        df = pd.read_sql(sample_query, engine)
        print("✓ Query executed successfully!")
        print("\nResults:")
        print(df)
        print(f"\nDataFrame shape: {df.shape}")
        print(f"Data types:\n{df.dtypes}")
        
except Exception as e:
    print(f"✗ Query execution failed: {e}")
    print("\nNote: This example query may fail if the tables don't exist.")
    print("Modify the query to match your database schema.")

✓ Query executed successfully!

Results:
  query_date query_time environment_name
0   26/01/13   20:35:41             test

DataFrame shape: (1, 3)
Data types:
query_date          object
query_time          object
environment_name    object
dtype: object


## 6. Usage Examples

### Example 1: Simple Connection with Context Manager

```python
from src.connection import TeradataConnection
import pandas as pd

conn_mgr = TeradataConnection()

# Query the test environment
with conn_mgr.get_connection('test') as engine:
    df = pd.read_sql("SELECT * FROM your_table", engine)
    print(df)
```

### Example 2: Switch Between Test and Prod

```python
# For development/testing
with conn_mgr.get_connection('test') as engine:
    test_data = pd.read_sql("SELECT COUNT(*) FROM table", engine)

# For production
with conn_mgr.get_connection('prod') as engine:
    prod_data = pd.read_sql("SELECT COUNT(*) FROM table", engine)
```

### Example 3: List Available Environments

```python
environments = conn_mgr.list_environments()
print(f"Available environments: {environments}")
```

### Example 4: Close All Connections

```python
# When done with all operations
conn_mgr.close_all()
```

## 6a. Using PDCRInfoReport for System Information

The `PDCRInfoReport` class provides convenient access to Teradata system information from DBC.DBCInfoV.


In [6]:
try:
    # Import the PDCRInfoReport class
    from src.reports import PDCRInfoReport
    
    # Initialize the report generator
    pdcr_report = PDCRInfoReport()
    print("✓ PDCRInfoReport initialized successfully")
    
    # Get PDCR info data from DBC.DBCInfoV
    info_df = pdcr_report.get_dbcinfo("test")
    
    print(f"\n✓ Retrieved PDCR info data: {len(info_df)} rows")
    print("\nFirst few rows:")
    print(info_df.head(10))
    
    # Show some statistics
    print(f"\nUnique InfoKey values: {info_df['InfoKey'].nunique()}")
    print("\nSample InfoKey values:")
    print(info_df['InfoKey'].unique()[:5])
    
    # Clean up
    pdcr_report.close()
    print("\n✓ PDCRInfoReport connections closed")
    
except Exception as e:
    print(f"✗ Error retrieving PDCR info: {e}")
    print("\nNote: This example requires a valid Teradata connection.")
    print("Ensure your td_env.yaml has correct credentials for the 'test' environment.")


2026-01-13 20:35:41,494 - src.connection - INFO - Loaded configuration for: ['test', 'prod']
2026-01-13 20:35:41,496 - src.reports - INFO - Executing PDCR info query on 'test' environment
2026-01-13 20:35:41,496 - src.connection - INFO - Connection string: teradatasql://@teradw/ALL?logmech=BROWSER


✓ PDCRInfoReport initialized successfully
Connection string: teradatasql://@teradw/ALL?logmech=BROWSER


2026-01-13 20:35:44,436 - src.connection - INFO - Created connection to 'test' environment
2026-01-13 20:35:45,077 - src.reports - INFO - Retrieved 3 rows from DBC.DBCInfoV
2026-01-13 20:35:45,171 - src.connection - INFO - Closed connection to 'test'



✓ Retrieved PDCR info data: 3 rows

First few rows:
                 InfoKey     InfoData
0  LANGUAGE SUPPORT MODE     Standard
1                RELEASE  17.20.03.34
2                VERSION  17.20.03.34

Unique InfoKey values: 3

Sample InfoKey values:
['LANGUAGE SUPPORT MODE' 'RELEASE' 'VERSION']

✓ PDCRInfoReport connections closed


## 7. Close the Connection

Properly clean up connections when done.

In [7]:
try:
    # Close all connections
    if 'conn_manager' in locals():
        conn_manager.close_all()
        print("✓ All connections closed successfully")
    else:
        print("No connection manager to close")
        
except Exception as e:
    print(f"✗ Error closing connections: {e}")

2026-01-13 20:35:45,261 - src.connection - INFO - Closed connection to 'test'


✓ All connections closed successfully
